In [8]:
from seedtools import show_all_datasets ,load_seed,register_csv
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten
import keras_tuner as kt 

In [20]:
ss =  StandardScaler()

In [21]:
data = pd.read_csv(load_seed("pima_indians_diabetest.csv"))
x =  data.drop(columns=["Outcome"]).values
y =  data["Outcome"].values
x_train,x_test,y_train,y_test =  train_test_split(x,y,test_size=0.2)

x_train =  ss.fit_transform(x_train)
x_test = ss.transform(x_test)

- data File ✔
- Seed file ✔
shape :  [768, 9]
columns :  ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
desc :  DATA IS NOT YET PROVIDED


**SIMPLE MODEL**

In [6]:
model = Sequential([
    Dense(32,activation="relu",input_dim=8),
    Dense(1,activation="sigmoid")
])
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["acc"])
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
history =  model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10


20/20 [==============================] - 1s 9ms/step - loss: 0.6616 - acc: 0.6319 - val_loss: 0.6597 - val_acc: 0.6494
Epoch 2/10
20/20 [==============================] - 0s 3ms/step - loss: 0.6245 - acc: 0.6759 - val_loss: 0.6284 - val_acc: 0.6753
Epoch 3/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5962 - acc: 0.7117 - val_loss: 0.6036 - val_acc: 0.7143
Epoch 4/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5735 - acc: 0.7199 - val_loss: 0.5839 - val_acc: 0.7143
Epoch 5/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5555 - acc: 0.7378 - val_loss: 0.5681 - val_acc: 0.7143
Epoch 6/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5388 - acc: 0.7394 - val_loss: 0.5538 - val_acc: 0.7143
Epoch 7/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5252 - acc: 0.7524 - val_loss: 0.5412 - val_acc: 0.7208
Epoch 8/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5147 - acc: 0.

**KERAS TUNER IMPLEMENTTATION**
- we can find anything like `loss`,`optimizer`,`num_layers`,`units in num_layers`,`activation`
- below is just the sample demonstration of optimizer finding 
- while running it a directory named `untitled_project` will be generated but yeah u can change its name 

In [22]:
def build_model(hp):
    model = Sequential([
        Dense(32,activation="relu",input_dim=8),
        Dense(1,activation="sigmoid")
    ])
    
    opt =  hp.Choice("optimizer",values=["adam","rmsprop","adadelta","sgd"])
    model.compile(loss="binary_crossentropy",optimizer=opt,metrics=["acc"])
    return model

In [23]:
tuner =  kt.RandomSearch(build_model,max_trials=5,objective="val_acc")

Reloading Tuner from .\untitled_project\tuner0.json


In [24]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [25]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [26]:
model =  tuner.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
model.fit(x_train,y_train,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 [==============================] - 0s 8ms/step - loss: 0.5312 - acc: 0.7459 - val_loss: 0.5564 - val_acc: 0.6948
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5142 - acc: 0.7590 - val_loss: 0.5436 - val_acc: 0.7078
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5007 - acc: 0.7622 - val_loss: 0.5340 - val_acc: 0.7143
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4908 - acc: 0.7704 - val_loss: 0.5272 - val_acc: 0.7078
Epoch 11/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4822 - acc: 0.7720 - val_loss: 0.5218 - val_acc: 0.7143
Epoch 12/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4758 - acc: 0.7736 - val_loss: 0.5186 - val_acc: 0.7208
Epoch 13/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4707 - acc: 0.7769 - val_loss: 0.5157 - val_acc: 0.7273
Epoch 14/100
20/20 [==============================] - 0s 2ms/step - loss: 0.466

In [28]:
loss,acc=  model.evaluate(x_test,y_test)
acc

5/5 [==============================] - 0s 990us/step - loss: 0.4979 - acc: 0.7532


0.7532467246055603